<code>
|=========================> import libs & modules:
</code>

In [15]:
import pandas as pd

<code>
|=========================> data frame for movies & users:
</code>

In [16]:


def openFile(file_path, mode='r'):
    try:
        return open(file_path, mode)
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        raise

fileGenre = openFile('../data/u.genre')
fileItem = openFile('../data/u.item')
fileUser = openFile('../data/u.user')
fileData = openFile('../data/u.data')

def getDataframeFromFile(filed):
    data = []
    #
    while True:

        line = filed.readline()

        # if line is empty
        if line.strip() == "":
            break

        rec = line.strip().split('|')
        


        item = {}

        if len(rec) > 6:
            item['movieId'] = rec[0]
            item['name'] = rec[1]
            item['release_date'] = rec[2]
            item['url'] = rec[4]

            #5 => 23
            item['genre'] = [genre[idx]['name'] for  idx, g in enumerate(range(5,24)) if rec[g] == '1']

        elif len(rec) == 2:
            item['id'] = rec[1]
            item['name'] = rec[0]
            
        elif len(rec) == 5:
            item['userId'] = rec[0]
            item['age'] = rec[1]
            item['gender'] = rec[2]
            item['function'] = rec[3]
            item['number'] = rec[4]
        else:
            print("Invalid record format. Skipping.")
            return

        data.append(item)
    filed.close()
    return data

genre = getDataframeFromFile(fileGenre)
items = getDataframeFromFile(fileItem)
users = getDataframeFromFile(fileUser)


df_movies = pd.DataFrame(items)
df_users = pd.DataFrame(users)

<code>
|=========================> data frame for relationship between movies & users:
</code>

In [17]:
fileData = open('../data/u.data', 'r')
def getDataframeFromFileWithSpaces(filed):
    data = []
    #
    while True:

        line = filed.readline()

        # if line is empty
        if line.strip() == "":
            break

        rec = line.strip().split('\t')

        item = {}
        
        item['userId'] = rec[0]
        item['movieId'] = rec[1]
        item['rating'] = rec[2]
        item['timestamp'] = rec[3]
        data.append(item)
    filed.close()
    return data

relationship = getDataframeFromFileWithSpaces(fileData)
df_relations = pd.DataFrame(relationship)

```
|=========================> merge dataframes ( users,ratings,movies ):
```

In [18]:
merged_df = pd.merge(df_movies, df_relations[['userId', 'movieId', 'rating','timestamp']], on='movieId', how='inner')
df = pd.merge(merged_df,df_users,on=['userId'],how='inner')

```
|=========================>  Note: the expected output :


{"userId": "365",
  "movie": 
    { "movieId": "7526",
        "title": "Lord of the Rings: The Fellowship of the Ring, The (2001)",
        "genres": ["Adventure", "Fantasy"] 
    },
    "rating": "5",
    "timestamp": "45554125",
    "userId": "13",
    "age": 19,
    "gender": "F",
    "functions": "45554125",
}
Ouput:
```

```
|=========================> save data on json:
```

In [19]:
res = df.to_json(orient='records',path_or_buf="../data/movies.json")
# df_movies.to_json(orient='records',path_or_buf="../data/movies.json")
# df_users.to_json(orient='records',path_or_buf="../data/users.json")
# df_relations.to_json(orient='records',path_or_buf="../data/ratings.json")